# 단어 분리하기( Byte Pair Encoding, BPE )

#### 기계가 학습 단계에서 학습한 단어들을 모아놓은 것을 단어 집합( vocabulary )라 한다. 그리고 테스트 단계에서 기계가 미처 배우지 못한 모르는 단어가 등장한다면 단어 집합에 없는 단어란 의미로 OOV( Out-Of-Vocabulary )가 한다. 또는 이를 UNK( Unknown Token )라 표현하기도 한다.

#### 기계가 문제를 풀 때, 모르는 단어가 등장하면 주어진 문제를 푸는 것이 훨씬 어려워진다. 이와 같이 모르는 단어로 인해 문제를 제대로 풀지 못하는 상황을 OOV 문제라 한다.

#### 단어 분리( Subword segmentation ) 작업은 하나의 단어는 의미있는 여러 내부 단어들( subwords )의 조합으로 구성된 경우가 많기 때문에 단어를 여러 단어로 분리해서 단어를 이해해보겠다는 의도를 가진 전처리 작업이다.

#### 언어의 특성에 따라 영어권 언어나 한국어는 단어 분리를 시도했을 때 어느 정도 의미있는 단위로 나누는 것이 가능하다.( 분리 토크나이저 )

#### 단어 분리는 기계가 아직 배운 적이 없는 단어에 대해 어느 정도 대처할 수 있도록 하며, 기계 번역 등에서 주요 전처리로 사용되고 있다. 

#### OOV 문제를 완화하는 대표적 단어 분리 토크나이저인 BPE( Byte Pair Endcoding ) 알고리즘과 실무에서 사용할 수 있는 단어 분리 토크나이저 구현체인 센텐스피스( Sentencepiece )

## 1. BPE( Byte Pair Encoding )

#### BPE( Byte Pair Encoding ) 알고리즘은 1994년에 제안된 데이터 압축 알고리즘이다. 하지만 후에 자연어 처리의 단어 분리 알고리즘으로 응용되었다.

#### 'aaabdaaabac' 문자열이 있을때 BPE는 기본적으로 연속적으로 가장 많이 등장한 글자의 쌍을 찾아서 하나의 글자로 병합하는 방식을 수행한다. 태생이 압축 알고리즘인 만큼, 여기서는 글자 대신 byte라 표현했을때 가장 자주 등장하고 있는 byte의 쌍( byte pair )은 'aa'이다. 이 'aa'라는 byte 쌍을 하나의 byte인 'Z'로 치환해 보겠다.

#### 'ZabdZabac'
#### Z = aa
#### 이제 가장 많이 등장하고 있는 byte 쌍은 'ab'입니다. 'ab'를 'Y'로 치환
#### ZYdZYac
#### Y = ab
#### Z = aa

#### 이제 가장 많이 등장하고 있는 byte 쌍은 'ZY'입니다. 이를 'X'로 치환
#### XdXac
#### X = ZY
#### Y = ab
#### Z = aa
#### 이제 더 이상 병합할 byte 쌍은 없으므로 BPE는 위의 결과를 최종 결과로 하여 종료된다.

## 2. 자연어 처리에서의 BPE( Byte Pair Encoding )

#### 자연어 처리에서의 BPE는 단어 분리( word segmentation ) 알고리즘이다. 기존에 있던 단어를 분리한다는 의미이다. 

BPE를 요약하면 글자( character ) 단위에서 점차적으로 단어 집합( vocabulary )을 만들어 내는 Bottom up 방식의 접근을 사용한다. 우선 훈련 데이터에 있는 단어들을 모든 글자( character ) 또는 유니코드( unicode ) 단위로 단어 집합( vocabulary )를 만들고, 가장 많이 등장하는 유니그램을 하나의 유니그램으로 통합한다.

### 1) 기존의 접근

#### 어떤 훈련 데이터로부터 각 단어들의 빈도수를 카운트했다고 했을때, 각 단어와 각 단어의 빈도수가 기록되어져 있는 해당 결과는 임의로 dictionary란 이름을 붙였다.
#### dictionary
#### 훈련 데이터에 있는 단어와 등장 빈도수
#### low : 5, lower : 2, newest : 6, widest : 3

#### 이 훈련 데이터에는 'low'란 단어가 5회 등장하였고, 'lower'란 단어는 2회 등장하였으며, 'newest'란 단어는 6회, 'widest'란 단어는 3회 등장하였다는 의미입니다.
#### 그렇다면 dictionary로부터 이 훈련 데이터의 단어 집합( vocabulary )을 얻는 것은 간단하다.
#### vocabulary
#### low, lower, newest, widest

#### 단어 집합은 중복을 배제한 단어들의 집합을 의미하므로 기존에 단어 집합의 정의라면, 이 훈련 데이터의 단어 집합에는 'low', 'lower', 'newest', 'widest'라는 4개의 단어가 존재한다. 그리고 이 경우 테스트 과정에서 'lowest'란 단어가 등장한다면 기계는 이 단어를 학습한 적이 없으므로 해당 단어에 대해서 제대로 대응하지 못하는 OOV 문제가 발생한다.

### 2) BPE 알고리즘을 사용한 경우

#### 위의 dictionary에 BPE를 적용, 우선 dictionary의 모든 단어들을 글자( character ) 단위로 분리한다. 이제부터 dictionary는 자신 또한 업데이트되며 앞으로 단어 집합을 업데이트하기 위해 지속적으로 참고되는 참고 자료 역활을 한다.

#### dictionary
#### low : 5, lower : 2, newest : 6, widest : 3

#### dictionary를 참고로 한 초기 단어 집합( vocabulary ), 초기 구성은 글자 단위로 분리된 상태이다.
#### vocabulary
#### l, o, w, e, r, n, w, s, t, i, d

#### BPE의 특징은 알고리즘의 동작을 몇 회 반복( iteration )할 것인지를 사용자가 정한다는 점이다. 
#### 가장 빈도수가 높은 유니그램의 쌍을 하나의 유니그램으로 통합하는 과정을 몇 회 반복할 것인지를 정한다.

#### BPE 동작을 10회 한다고 가정했을때

#### 1회 - dictionary를 참고로 하였을 때 빈도수가 9로 가장 높은 ( e, s )의 쌍을 es로 통합한다.
#### dictionary update
#### l o w : 5,
#### l o w e r : 2,
#### n e w es t : 6,
#### w i d es t : 3

#### vocabulary update
#### l, o, w, e, r, n, w, s, t, i, d, es

#### 2회 - 빈도수가 9로 가장 높은 ( es, t )의 쌍을 est로 통합한다.
#### dictionary update
#### l o w : 5,
#### l o w e r : 2,
#### n e w est : 6,
#### w i d est : 3
#### vocabulary update
#### l, o, w, e, r, n, w, s, t, i, d, es, est

#### 3회 - 빈도수가 7로 가장 높은 ( l, o )의 쌍을 lo로 통합한다.
#### dictionary update
#### lo w : 5,
#### lo w e r : 2,
#### n e w est : 6,
#### w i d est : 3
#### vocabulary update
#### l, o, w, e, r, n, w, s, t, i, d, es, est, lo

#### 이와 같은 방식으로 총 10회 반복하였을 때 얻은 dictionary와 단어 집합은 다음과 같다.
#### dictionary update
#### low : 5,
#### low e r : 2,
#### newest : 6,
#### widest : 3
#### vocabulary update
#### l, o, w, e, r, n, w, s, t, i, d, es, est, lo, low, ne, new, newest, wi, wid, widest

#### 이 경우 테스트 과정에서 'lowest'란 단어가 등장한다면, 기존에는 OOV에 해당되는 단어가 되었겠지만 BPE 알고리즘을 사용한 위으 단어 집합에서는 더 이상 'lowest'는 OOV가 아니다. 기계는 우선 'lowest'를 전부 글자 단위로 분할한다. 즉 'l, o, w, e, s, t'가 된다. 그리고 기계는 위의 단어 집합을 참고로 하여 'low'와 'est'를 찾아낸다. 즉 'lowest'를 기계는 'low'와 'est' 두 단어로 인코딩한다. 그리고 이 두 단어는 둘 다 단어 집합에 있는 단어이므로 OOV가 아니다.

### 3) 코드 실습

In [5]:
import re, collections

#### BPE를 몇 회 수행할 것인지를 정한다.

In [9]:
num_merges = 10

#### BPE에 사용할 단어가 low, lower, newest, widest 일 때, BPE의 입력으로 사용하는 실제 단어 집합. </w>는 단어의 맨 끝에 붙는 특수 문자이며, 각 단어는 글자( character ) 단위로 분리한다.

In [7]:
vocab = { 'l o w </w>' : 5,
          'l o w e r </w>' : 2,
          'n e w e s t </w>' : 6,
          'w i d e s t </w>' : 3 
        }

#### BPE 코드는 가장 빈도수가 높은 유니그램의 쌍을 하나의 유니그램으로 통합하는 과정을 num_merges회 반복한다.

In [11]:
def get_stats( vocab ):
    pairs = collections.defaultdict( int )
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range( len( symbols ) - 1 ):
            pairs[ symbols[ i ], symbols[ i + 1 ] ] += freq
    
    return pairs

def merge_vocab( pair, v_in ):
    v_out = {}
    bigram = re.escape( ' '.join( pair ) )
    p = re.compile( '(?<!\S)' + bigram + '(?!\S)' )
    for word in v_in:
        w_out = p.sub( ''.join( pair ), word )
        v_out[ w_out ] = v_in[ word ]
        
    return v_out

for i in range( num_merges ):
    pairs = get_stats( vocab )
    best = max( pairs, key = pairs.get )
    vocab = merge_vocab( best, vocab )
    print( best )

('e', 's')
('es', 't')
('est', '</w>')
('l', 'o')
('lo', 'w')
('n', 'e')
('ne', 'w')
('new', 'est</w>')
('low', '</w>')
('w', 'i')


#### 출력 결과는 글자들의 통합 과정을 보여주고 있다. 'e'와 's'의 쌍은 초기 단어 집합에서 총 9회 등장했다. 그렇기 때문에 'es'로 통합된다. 그 다음으로는 'es'와 't'의 쌍을, 그 다음으로는 'est'와 '</w>' 쌍을 통합 시킨다. 빈도수가 가장 높은 순서대로 통합하는 이 과정을 총 num_merges회 반복한다.

## WPM( Wordpiece Model )

#### BPE 외에도 WPM( Wordpiece Model )이나 Unigram Language Model Tokenizer와 같은 단어 분리 토크나이저들이 존재한다.
#### 구글은 WPM을 일종의 BPE의 변형으로 소개한다. 
#### WPM은 BPE와 달리 빈도수가 아니라 우도( likelihood )를 통해서 단어를 분리한다. 

#### WPM을 수행하기 이전 문장 : Jet makers feud over seat width with big orders at stake
#### WPM을 수행한 결과( wordpieces ) : _J et _makers _fe ud _over _seat _width _with _big _orders _at _stake

#### 'Jet'는 'J'와 'et'로 나누어졌으며, 'feud'는 'fe'와 'ud'로 나누어진 것을 볼 수 있다. WPM은 모든 단어의 맨 앞에 _를 붙이고, 단어는 내부단어( subword )로 통계에 기반하여 띄어쓰기로 분리한다. '_( underbar )'는 분장 복원을 위한 장치이다. WPM이 수행된 결과로부터 다시 수행 전의 결과로 돌리는 방법은 현재 있는 모든 띄어쓰기를 전부 제거하고, 언더바를 띄어쓰기로 바꾸면 된다.

#### 구글의 WPM은 Open Source로 공개되지 않았다

## 3. 센텐스피스( Sentencepiece )

#### 실무에서 단어 분리를 위해서 어떤 구현체를 사용해야 하냐고 묻는다면, 구글의 센텐스피스( Sentencepiece )를 사용한다.
#### 구글은 BPE 알고리즘과 Unigram Language Model Tokenizer를 구현한 센텐스피스를 Github에 공개하였다. 기존의 BPE 알고리즘도 Github에 공개하기는 했지만, 이를 실무에 사용하기에는 속도가 매우 느리므로 센텐스피스를 사용하는 것을 권장

#### 센텐스피스의 이점은 단어 분리 알고리즘을 사용하기 위해서, 데이터에 있는 단어 토큰화를 먼저 진행한 상태여야 한다면 이 단어 분리 알고리즘을 도든 언어에 사용하는 것은 쉽지 않다. 영어와 달리 한국어와 같은 언어는 단어 토큰화부터가 쉽지 않기 때문이다. 

#### 그런데 이런 사전 토큰화 작업( pretokenization )없이 전처리를 하지 않은 데이터( raw data )에 바로 단어 분리 토크나이저를 사용할 수 있다면, 이 토크나이저는 그 어떤 언어에도 적용할 수 있는 토크나이저가 될 것이다. 

#### 센텐스피스는 이 이점을 살려서 구현되었다. 사전 토큰화 작업없이 단어 분리 토큰화를 수행하므로 언어에 종속되지 않는다.